In [2]:
# Cobain dari youtube
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [19]:
driver = webdriver.Chrome()

# Open the Tokopedia URL
url = 'https://www.tokopedia.com/search?st=&q=nugget%20ikan&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource='
driver.get(url)

data = []

for i in range(200) : 
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#zeus-root")))
    time.sleep(2)
    
    for j in range (21): 
        driver.execute_script("window.scrollBy(0, 250)")
        time.sleep(1)

    driver.execute_script("window.scrollBy(50, 0)")
    time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    products = soup.find_all('div', class_="css-5wh65g")

    # Extract product information
    for product in products:
        # Extract product name
        product_name_tag = product.find('span', class_='_0T8-iGxMpV6NEsYEhwkqEg==')
        product_name = product_name_tag.text.strip() if product_name_tag else 'N/A'

        # Extract price
        price_tag = product.find('div', class_='_67d6E1xDKIzw+i2D2L0tjw==')
        price = price_tag.text.strip() if price_tag else 'N/A'

        # Extract shop name
        shop_name_tag = product.find('span', class_='pC8DMVkBZGW7-egObcWMFQ== flip')
        shop_name = shop_name_tag.text.strip() if shop_name_tag else 'N/A'

        # Extract rating
        rating_tag = product.find('span', class_='_9jWGz3C-GX7Myq-32zWG9w==')
        rating = rating_tag.text.strip() if rating_tag else 'N/A'

        # Extract number of items sold
        sold_tag = product.find('span', class_='se8WAnkjbVXZNA8mT+Veuw==')
        sold = sold_tag.text.strip() if sold_tag else 'N/A'

        # Store the data in a set to avoid duplicates
        data.append(
            (product_name, price, shop_name, rating, sold)
        )
    
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, "button[aria-label='Laman berikutnya']").click()
    time.sleep(4)


df = pd.DataFrame(data, columns = ['Product_name', 'price', 'shop_name', 'rating', 'sold'])

driver.quit()